In [1]:
import sys
import warnings
warnings.filterwarnings("ignore")
import numpy as np
np.set_printoptions(threshold=sys.maxsize) #print out full arrays
import pandas as pd

randomstate = 42

from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole

import get_properties_functions_for_WI as gp

import importlib
import sys

In [2]:
prefix = "pyrd"

file_name = prefix + "_atom_map.xlsx"

atom_map_df = pd.read_excel(
    file_name, "Sheet1", index_col=0, header=0, engine="openpyxl"
)

display(atom_map_df.head())

df = atom_map_df  # df is what properties will be appended to, this creates a copy so that you have the original preserved

,log_name,C3,C4,C5,N1,C1,C2
0,pyrd10_conf-1,C10,C5,C6,N7,C8,C9
1,pyrd10_conf-10,C10,C5,C6,N7,C8,C9
2,pyrd10_conf-11,C10,C5,C6,N7,C8,C9
3,pyrd10_conf-12,C10,C9,C8,N7,C6,C5
4,pyrd10_conf-13,C10,C5,C6,N7,C8,C9


In [3]:
# testing subprocess with goodvibes

importlib.reload(gp)
import subprocess

log_file = "pyrd1_conf-1.log"

# Construct command-line arguments for the new version of goodvibes
cmd_args = [
    sys.executable, "-m",
    "goodvibes", 
    log_file,
    "--spc", "link",
    "-t", str(298.15)
]

# Run the goodvibes command and capture the output
result = subprocess.run(cmd_args, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# print the output
print(result.stdout)

   GoodVibes v3.2 2024/08/25 19:48:58
   Citation: Luchini, G.; Alegre-Requena, J. V.; Funes-Ardoiz, I.; Paton, R. S. F1000Research, 2020, 9, 291.
   GoodVibes version 3.2 DOI: 10.12688/f1000research.22758.1

o  Requested: --spc link -t 298.15 

   Temperature = 298.15 Kelvin   Pressure = 1 atm
   All energetic values below shown in Hartree unless otherwise specified.

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Entropic quasi-harmonic treatment: frequency cut-off value of 100.0 wavenumbers will be applied.
   QS = Grimme: Using a mixture of RRHO and Free-rotor vibrational entropies.
   REF: Grimme, S. Chem. Eur. J. 2012, 18, 9955-9964

   Combining final single point energy with thermal corrections.

   Structure                                       E_SPC             E        ZPE         H_SPC        T.S     T.qh-S      G(T)_SPC   qh-G(T)_SPC
   *********************************************************************************************************

In [4]:
# this box has functions to choose from
df = atom_map_df

# ---------------GoodVibes Engergies---------------
# uses the GoodVibes 2021 Branch (Jupyter Notebook Compatible)
# calculates the quasi harmonic corrected G(T) and single point corrected G(T) as well as other thermodynamic properties
# inputs: dataframe, temperature
df = gp.get_goodvibes_e(df, 298.15)

# ---------------Frontier Orbitals-----------------
# E(HOMO), E(LUMO), mu(chemical potential or negative of molecular electronegativity), eta(hardness/softness), omega(electrophilicity index)
df = gp.get_frontierorbs(df)

# ---------------Polarizability--------------------
# Exact polarizability
df = gp.get_polarizability(df)

# ---------------Dipole----------------------------
# Total dipole moment magnitude in Debye
df = gp.get_dipole(df)

# ---------------Volume----------------------------
# Molar volume
# requires the Gaussian keyword = "volume" in the .com file
df = gp.get_volume(df)

# ---------------SASA------------------------------
# Uses morfeus to calculat sovlent accessible surface area and the volume under the SASA
df = gp.get_SASA(df)

# ---------------NBO-------------------------------
# natural charge from NBO
# requires the Gaussian keyword = "pop=nbo7" in the .com file
nbo_list = ["C1", "C2", "C3", "C4", "C5", "N1"]
df = gp.get_nbo(df, nbo_list)

# ---------------NMR-------------------------------
# isotropic NMR shift
# requires the Gaussian keyword = "nmr=giao" in the .com file
nmr_list = ["C1", "C2", "C3", "C4", "C5", "N1"]
df = gp.get_nmr(df, nmr_list)

# ---------------Distance--------------------------
# distance between 2 atoms
dist_list_of_lists = [["N1", "C1"], ["N1", "C5"]]
df = gp.get_distance(df, dist_list_of_lists)

# ---------------Angle-----------------------------
# angle between 3 atoms
angle_list_of_lists = [["C5", "N1", "C1"]]
df = gp.get_angles(df, angle_list_of_lists)

# ---------------Dihedral--------------------------
# dihedral angle between 4 atoms
dihedral_list_of_lists = [["C4", "C5", "N1", "C1"], ["C2", "C1", "N1", "C5"]]
df = gp.get_dihedral(df, dihedral_list_of_lists)

# ---------------Vbur Scan-------------------------
# uses morfeus to calculate the buried volume at a series of radii (including hydrogens)
# inputs: dataframe, list of atoms, start_radius, end_radius, and step_size
# if you only want a single radius, put the same value for start_radius and end_radius (keep step_size > 0)
vbur_list = ["C1", "C2", "C3", "C4", "C5", "N1"]
df = gp.get_vbur_scan(df, vbur_list, 2, 4, 0.5)

# ---------------Sterimol morfeus------------------
# uses morfeus to calculate Sterimol L, B1, and B5 values
# NOTE: this is much faster than the corresponding DBSTEP function (recommendation: use as default/if you don't need Sterimol2Vec)
sterimol_list_of_lists = [["N1", "C1"], ["N1", "C5"]]
df = gp.get_sterimol_morfeus(df, sterimol_list_of_lists)

# ---------------Buried Sterimol-------------------
# uses morfeus to calculate Sterimol L, B1, and B5 values within a given sphere of radius r_buried
# atoms outside the sphere + 0.5 vdW radius are deleted and the Sterimol vectors are calculated
# for more information: https://kjelljorner.github.io/morfeus/sterimol.html
# inputs: dataframe, list of atom pairs, r_buried
sterimol_list_of_lists = [["N1", "C1"], ["N1", "C5"]]
df = gp.get_buried_sterimol(df, sterimol_list_of_lists, 5.5)

# ---------------Sterimol DBSTEP-------------------
# uses DBSTEP to calculate Sterimol L, B1, and B5 values
# default grid point spacing (0.05 Angstrom) is used (can use custom spacing or vdw radii in the get_properties_functions script)
# more info here: https://github.com/patonlab/DBSTEP
# NOTE: this takes longer than the morfeus function (recommendation: only use this if you need Sterimol2Vec)
sterimol_list_of_lists = [["N1", "C1"], ["N1", "C5"]]
df = gp.get_sterimol_dbstep(df, sterimol_list_of_lists)

# ---------------Sterimol2Vec----------------------
# uses DBSTEP to calculate Sterimol Bmin and Bmax values at intervals from 0 to end_radius, with a given step_size
# default grid point spacing (0.05 Angstrom) is used (can use custom spacing or vdw radii in the get_properties_functions script)
# more info here: https://github.com/patonlab/DBSTEP
# inputs: dataframe, list of atom pairs, end_radius, and step_size
sterimol2vec_list_of_lists = [["N1", "C1"], ["N1", "C5"]]
df = gp.get_sterimol2vec(df, sterimol2vec_list_of_lists, 1, 1.0)

# ---------------Pyramidalization------------------
# uses morfeus to calculate pyramidalization based on the 3 atoms in closest proximity to the defined atom
# collects values based on two definitions of pyramidalization
# details on these values can be found here: https://kjelljorner.github.io/morfeus/pyramidalization.html
pyr_list = ["C1", "C2", "C3", "C4", "C5", "N1"]
df = gp.get_pyramidalization(df, pyr_list)

# ---------------Plane Angle-----------------------
# !plane angle between 2 planes (each defined by 6 atoms)
# planeangle_list_of_lists = [["N1", "C1", "C5"], ["C2", "C3", "C4"]]
# df = gp.get_planeangle(df, planeangle_list_of_lists)

# --------------LP energy - custom from first cell---------------
lp_list = ["N1"]
df = gp.get_one_lp_energy(df, lp_list)

# ---------------Time----------------------------------
# returns the total CPU time and total Wall time (not per subjob) because we are pioneers
# if used in summary df, will give the average (not Boltzmann average) in the Boltzmann average column
df = gp.get_time(df)

# ---------------ChelpG----------------------------
# ChelpG ESP charge
# requires the Gaussian keyword = "pop=chelpg" in the .com file
a_list = ["C1", "C2", "C3", "C4", "C5", "N1"]
df = gp.get_chelpg(df, a_list)

# ---------------Hirshfeld-------------------------
# Hirshfeld charge, CM5 charge, Hirshfeld atom dipole
# requires the Gaussian keyword = "pop=hirshfeld" in the .com file
a_list = ["C1", "C2", "C3", "C4", "C5", "N1"]
df = gp.get_hirshfeld(df, a_list)

pd.options.display.max_columns = None
display(df)

Goodvibes function has completed
Frontier orbitals function has completed
Polarizability function has completed
Dipole function has completed
Volume function has completed
SASA function has completed
NBO function has completed for ['C1', 'C2', 'C3', 'C4', 'C5', 'N1']
NMR function has completed for ['C1', 'C2', 'C3', 'C4', 'C5', 'N1']
Distance function has completed for [['N1', 'C1'], ['N1', 'C5']]
Angles function has completed for [['C5', 'N1', 'C1']]
Dihedral function has completed for [['C4', 'C5', 'N1', 'C1'], ['C2', 'C1', 'N1', 'C5']]
Vbur scan function has completed for ['C1', 'C2', 'C3', 'C4', 'C5', 'N1'] from 2 to 4
Morfeus Sterimol function has completed for [['N1', 'C1'], ['N1', 'C5']]
Morfeus Buried Sterimol function has completed for [['N1', 'C1'], ['N1', 'C5']]
   pyrd10_conf-1.log / Bmin:  1.65 / Bmax:  6.45 / L:  4.25
   pyrd10_conf-1.log / Bmin:  1.65 / Bmax:  4.43 / L:  6.35
   pyrd10_conf-10.log / Bmin:  1.65 / Bmax:  8.93 / L:  4.25
   pyrd10_conf-10.log / Bmin:  1.65

,log_name,C3,C4,C5,N1,C1,C2,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,NBO_charge_C3,NBO_charge_C4,NBO_charge_C5,NBO_charge_N1,NMR_shift_C1,NMR_shift_C2,NMR_shift_C3,NMR_shift_C4,NMR_shift_C5,NMR_shift_N1,distance_N1_C1(Å),distance_N1_C5(Å),angle_C5_N1_C1(°),dihedral_C4_C5_N1_C1(°),dihedral_C2_C1_N1_C5(°),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,%Vbur_C3_2.0Å,%Vbur_C4_2.0Å,%Vbur_C5_2.0Å,%Vbur_N1_2.0Å,%Vbur_C1_2.5Å,%Vbur_C2_2.5Å,%Vbur_C3_2.5Å,%Vbur_C4_2.5Å,%Vbur_C5_2.5Å,%Vbur_N1_2.5Å,%Vbur_C1_3.0Å,%Vbur_C2_3.0Å,%Vbur_C3_3.0Å,%Vbur_C4_3.0Å,%Vbur_C5_3.0Å,%Vbur_N1_3.0Å,%Vbur_C1_3.5Å,%Vbur_C2_3.5Å,%Vbur_C3_3.5Å,%Vbur_C4_3.5Å,%Vbur_C5_3.5Å,%Vbur_N1_3.5Å,%Vbur_C1_4.0Å,%Vbur_C2_4.0Å,%Vbur_C3_4.0Å,%Vbur_C4_4.0Å,%Vbur_C5_4.0Å,%Vbur_N1_4.0Å,Sterimol_L_N1_C1(Å)_morfeus,Sterimol_B1_N1_C1(Å)_morfeus,Sterimol_B5_N1_C1(Å)_morfeus,Sterimol_L_N1_C5(Å)_morfeus,Sterimol_B1_N1_C5(Å)_morfeus,Sterimol_B5_N1_C5(Å)_morfeus,Buried_Sterimol_L_N1_C1_5.0(Å),Buried_Sterimol_B1_N1_C1_5.0(Å),Buried_Sterimol_B5_N1_C1_5.0(Å),Buried_Sterimol_L_N1_C5_5.0(Å),Buried_Sterimol_B1_N1_C5_5.0(Å),Buried_Sterimol_B5_N1_C5_5.0(Å),Sterimol_B1_N1_C1(Å)_dbstep,Sterimol_B5_N1_C1(Å)_dbstep,Sterimol_L_N1_C1(Å)_dbstep,Sterimol_B1_N1_C5(Å)_dbstep,Sterimol_B5_N1_C5(Å)_dbstep,Sterimol_L_N1_C5(Å)_dbstep,Sterimol_Bmin_N1_C1_0.0Å(Å),Sterimol_Bmax_N1_C1_0.0Å(Å),Sterimol_Bmin_N1_C1_1.0Å(Å),Sterimol_Bmax_N1_C1_1.0Å(Å),Sterimol_Bmin_N1_C5_0.0Å(Å),Sterimol_Bmax_N1_C5_0.0Å(Å),Sterimol_Bmin_N1_C5_1.0Å(Å),Sterimol_Bmax_N1_C5_1.0Å(Å),pyramidalization_Gavrish_C1(°),pyramidalization_Agranat-Radhakrishnan_C1,pyramidalization_Gavrish_C2(°),pyramidalization_Agranat-Radhakrishnan_C2,pyramidalization_Gavrish_C3(°),pyramidalization_Agranat-Radhakrishnan_C3,pyramidalization_Gavrish_C4(°),pyramidalization_Agranat-Radhakrishnan_C4,pyramidalization_Gavrish_C5(°),pyramidalization_Agranat-Radhakrishnan_C5,pyramidalization_Gavrish_N1(°),pyramidalization_Agranat-Radhakrishnan_N1,NBO_LP_occupancy_N1,NBO_LP_energy_N1,CPU_time_total(hours),Wall_time_total(hours),ChelpG_charge_C1,ChelpG_charge_C2,ChelpG_charge_C3,ChelpG_charge_C4,ChelpG_charge_C5,ChelpG_charge_N1,Hirsh_charge_C1,Hirsh_CM5_charge_C1,Hirsh_atom_dipole_C1,Hirsh_charge_C2,Hirsh_CM5_charge_C2,Hirsh_atom_dipole_C2,Hirsh_charge_C3,Hirsh_CM5_charge_C3,Hirsh_atom_dipole_C3,Hirsh_charge_C4,Hirsh_CM5_charge_C4,Hirsh_atom_dipole_C4,Hirsh_charge_C5,Hirsh_CM5_charge_C5,Hirsh_atom_dipole_C5,Hirsh_charge_N1,Hirsh_CM5_charge_N1,Hirsh_atom_dipole_N1
0,pyrd10_conf-1,C10,C5,C6,N7,C8,C9,-2939.790206,0.165409,-2939.614581,0.046949,0.045717,-2939.661530,-2939.660298,298.15,-0.31256,-0.00240,-0.157480,0.31016,0.03998,115.2740,54.7674,3.9559,1402.360,332.622291,490.744031,0.904557,0.02305,-0.25205,-0.17870,-0.06744,0.02280,-0.41008,12.3385,41.5624,26.7586,27.2832,8.3600,-155.3399,1.33817,1.33818,117.358,0.079,0.123,90.783187,92.239153,92.436080,97.507748,92.436080,80.123321,73.785649,75.651126,78.293072,90.837023,80.365282,66.847897,57.662582,59.864794,65.481600,81.398056,68.778866,55.674538,45.123812,47.400881,55.443417,69.576966,59.388977,46.001444,35.587875,37.657889,46.429070,56.381122,51.223543,38.440351,4.689072,1.700000,6.475518,6.803286,1.700000,4.442045,4.689072,1.700000,6.162405,6.504377,1.700000,4.442045,1.650000,6.453875,4.25,1.650000,4.429447,6.35,1.650000,6.296229,1.650000,5.881539,1.626346,4.429447,1.650000,4.385487,0.007243,1.087745e-03,0.020593,3.099534e-03,0.038120,5.739571e-03,0.150960,2.269814e-02,0.044915,6.739750e-03,8.297220,1.998491,1.91767,-0.37802,2.27845,0.145194,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data
1,pyrd10_conf-10,C10,C5,C6,N7,C8,C9,-2939.788079,0.164826,-2939.612616,0.048883,0.047

## Save collected properties to Excel

Helpful to save here in case the Notebook crashes or if you want to add more properties before post-processsing. Can be read in at 5.1.1.

In [5]:
writer = pd.ExcelWriter(prefix + "_extracted_properties.xlsx", engine="xlsxwriter")
df.to_excel(writer)
writer.close()

In [6]:
# save the pandas dataframe to a pickle file
df.to_pickle(prefix + "_extracted_properties.pkl")